In [ ]:
import os
import PyPDF2
import re
import openai
import csv
import time

In [ ]:
# Define a function to extract the introduction and discussion sections from a PDF file
def extract_sections(pdf_file):
    didnt_find = []
    # Open the PDF file
    with open(pdf_file, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        
        # Iterate through the pages of the PDF file and extract the text
        text = ''
        for i in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[i]
            text += page.extract_text()

        # Convert all the text to lowercase
        text = text.lower()

        # Use regular expressions to find the introduction and discussion sections
        intro_match = re.search(r'introduction', text)
        disc_match = re.search(r'discussion', text)

        # If the sections are found, extract the first 1500 tokens
        if intro_match and disc_match:
            intro_end = intro_match.end()
            disc_start = disc_match.start()
            intro_text = text[intro_end:disc_start]
            disc_text = text[disc_match.end():]
        
            # Take the first 1500 tokens of the text after the title section
            intro_tokens = intro_text.split()
            disc_tokens = disc_text.split()
            if len(intro_tokens) > 1500:
                intro_text = ' '.join(intro_tokens[:1500])
            if len(disc_tokens) > 1500:
                disc_text = ' '.join(disc_tokens[:1500])

            return intro_text, disc_text
        else:
            return 'DidntFind'
        

In [ ]:
# Define a function to ask OpenAI the questions and parse the responses
def ask_questions(text):
    results = []
    for question in questions:
        message1 = question + "\n" + text
        message2 = [{"role": "system", "content": "Climate change specialist"}, {"role": "user", "content": f"{message1}"}]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages= message2,
            temperature=0.1
        )
        answer = response.choices[0].message.content
        results.append(answer)
        time.sleep(1)
    return results

In [ ]:
# Set the path to the folder containing the PDF articles
pdf_folder = 'path'

# Set up OpenAI API credentials
openai.api_key = "Your API Key"

# Set up the questions to ask
questions = [
    "What is climate change?"
]

# Open the CSV file for writing
with open('results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(['File', 'Question', 'Answer'])

    # Loop through all the PDF files in the folder
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            pdf_file = os.path.join(pdf_folder, filename)
            print(filename)

            # Extract the introduction and discussion sections from the PDF file
            sections = extract_sections(pdf_file)
            if sections != 'DidntFind':
                intro_text, disc_text = sections
                print('Extracting the introduction and the discussion')

                # Ask OpenAI the questions and parse the responses
                intro_results = ask_questions(intro_text)
                disc_results = ask_questions(disc_text)
                print('Asking ChatGPT')

                # Write the results to the CSV file
                for i in range(len(questions)):
                    print('Saving')
                    writer.writerow([filename, questions[i], intro_results[i]])
                    writer.writerow([filename, questions[i], disc_results[i]])
            
            else:
                writer.writerow([filename, 'DidntFind', 'DidntFind'])
                print('Saving')
